In [1]:
import pandas as pd
import os
import requests
import numpy as np
import json

In [44]:
import tweepy

# Read in the config file with your personal keys, secrets, and tokens
with open('config_file.txt') as file:
    lines = file.readlines()
    consumer_key = lines[0].strip('\n')
    consumer_secret = lines[1].strip('\n')
    access_token = lines[2].strip('\n')
    access_secret = lines[3].strip('\n')
    
# Set up the API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [4]:
# Read in twitter archive
tweets = pd.read_csv('twitter-archive-enhanced.csv')

In [6]:
# Grab the image predictions
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
images = requests.get(url)

In [24]:
# Create tsv of images
with open('images.csv', 'x') as file:
    file.write(images.text)

In [27]:
images_df = pd.read_csv('images.csv', sep = '\t')

In [53]:
file_name = 'tweet_json.txt'
with open(file_name, 'x') as outfile:
    for tweet in tweets['tweet_id']:
        tweet_json = api.get_status(tweet)._json
        json.dump(tweet_json, outfile)
        outfile.write('\n')
